In [ ]:
import os
import shutil
import numpy as np
import metamaps

In [ ]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = 'metamaps'

# The directory where we will
# store our smaller dataset
base_dir = 'metamaps_input'

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

def show_number_of_files_in_dir(dirname):
    print("Num files in '{}': {}".format(dirname,len(os.listdir(dirname))))

In [ ]:
show_number_of_files_in_dir(original_dataset_dir)

# Reset input directory
shutil.rmtree(base_dir)

In [ ]:
def safe_create(dirname):
    if not os.path.exists(dirname):
        print("Creating '{}'".format(dirname))
        os.mkdir(dirname)
    else:
        print("'{}' already exists".format(dirname))

safe_create(base_dir)
safe_create(train_dir)
safe_create(validation_dir)
safe_create(test_dir)

In [ ]:
# Randomize the list of files
input_files = os.listdir(original_dataset_dir)
np.random.shuffle(input_files)

BUCKET_SIZE = 8500

def copy_bucket_of_maps(start_bucket,num_buckets,dest_dir):
    if len(os.listdir(dest_dir)) == 0:
        print("Copying {} files into '{}'".format(num_buckets*BUCKET_SIZE, dest_dir))
        start = start_bucket * BUCKET_SIZE
        end = (start_bucket + num_buckets) * BUCKET_SIZE
        for fname in input_files[start:end]:
            src = os.path.join(original_dataset_dir, fname)
            dst = os.path.join(dest_dir, fname)
            shutil.copyfile(src, dst)
    else:
        print("Skipped copying files into '{}'".format(dest_dir))

# Copy two buckets worth of metamaps images to train_dir
copy_bucket_of_maps(0,2,train_dir)

# Copy next bucket of metamaps to validation_cats_dir
copy_bucket_of_maps(2,1,validation_dir)
    
# Copy next bucket of metamaps to test_cats_dir
copy_bucket_of_maps(3,1,test_dir)

NUM_REAL_TRAIN_MAPS = 2 * BUCKET_SIZE
NUM_REAL_VALIDATION_MAPS = BUCKET_SIZE
NUM_REAL_TEST_MAPS = BUCKET_SIZE

In [ ]:
# Verify number of files in input directories
show_number_of_files_in_dir(train_dir)
show_number_of_files_in_dir(validation_dir)
show_number_of_files_in_dir(test_dir)

In [ ]:
#Create model and add layers
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64, 64, len(metamaps.EncodingDim))))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
BATCH_SIZE = 20

TRAINING_STEPS = int((NUM_REAL_TRAIN_MAPS * 2) / BATCH_SIZE)
VALIDATION_STEPS = int((NUM_REAL_VALIDATION_MAPS * 2) / BATCH_SIZE)

print("Training steps: {}".format(TRAINING_STEPS))
print("Validation steps: {}".format(VALIDATION_STEPS))

In [ ]:
train_sequence = metamaps.MetaMapsSequence(train_dir,BATCH_SIZE)
validation_sequence = metamaps.MetaMapsSequence(validation_dir,BATCH_SIZE)

In [ ]:
history = model.fit_generator(
      train_sequence,
      steps_per_epoch=TRAINING_STEPS, # number of batches to train with
      epochs=1, # this will get tweaked by looking at the overfitting curves
      validation_data=validation_sequence,
      validation_steps=VALIDATION_STEPS) # number of batches to validate with

In [ ]:
model.save('real_vs_fake_metamaps .h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()